In [3]:
import pmdarima as pm
import pickle
import pandas as pd

## import model_combined, model_arima_2016, model_prophet

In [ ]:
with open('model_combined.pkl', 'rb') as pkl:
    model_combined = pickle.load(pkl)
with open('model_arima_2016.pkl', 'rb') as pkl:
    model_arima_p = pickle.load(pkl)
    
from prophet.serialize import model_to_json, model_from_json

with open('model_prophet.json', 'r') as fin:
    model_prophet = model_from_json(fin.read())  # Load model


## import future data

In [30]:
df_future = pd.read_csv('df_future.csv', parse_dates=True, index_col=0)
df_future

,timestep,day_of_year_sin,day_of_year_cos,lag1,lag2,lag3,lag4,lag5
2022-10-01,53599.0,-0.999991,0.004304,-2.54998,-3.809203,-4.167506,-2.524842,-0.281163
2022-10-02,53600.0,-0.999769,0.021516,0.00000,-2.549980,-3.809203,-4.167506,-2.524842


In [31]:
ds = pd.date_range(start='2022-10-01', periods=2, freq='D')
df_future_prophet = pd.DataFrame(data = ds, columns=['ds'])
df_future_prophet

,ds
0,2022-10-01
1,2022-10-02


## compare prediction

In [48]:
model_combined.predict(df_future) 

array([11.25572979, 13.16078932])

In [49]:
# forecast for the next month
model_arima_p.predict(1)

2022-10-31    10.586408
Freq: M, dtype: float64

In [50]:
model_prophet.predict(df_future_prophet)[['ds','yhat']]

,ds,yhat
0,2022-10-01,13.904953
1,2022-10-02,13.770986


In [69]:
model_prophet.predict(df_future_prophet)[['yhat']].round(2)

,yhat
0,13.90
1,13.77


In [73]:
pred_combi = model_combined.predict(df_future).round(2)  
pred_arima = model_arima_p.predict(1).values[0].round(2) 
pred_prophet =  model_prophet.predict(df_future_prophet)[['yhat']].round(2)

print(f'''
Temperature predition from combined model: 
    2022-10-01 is {pred_combi[0]}
    2022-10-02 is {pred_combi[1]}

Temperature predition from Prophet model: 
    2022-10-01 is {pred_prophet.loc[0,'yhat']}
    2022-10-02 is {pred_prophet.loc[1,'yhat']}

Temperature predition from ARIMA model:
    2022-10 is {pred_arima}''')



Temperature predition from combined model: 
    2022-10-01 is 11.26
    2022-10-02 is 13.16

Temperature predition from Prophet model: 
    2022-10-01 is 13.9
    2022-10-02 is 13.77

Temperature predition from ARIMA model:
    2022-10 is 10.59
